In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retreiver = WikipediaRetriever(top_k_results=2, lang="en")

In [3]:
query = "the geopolitical history of india and pakistan from the perspective of a chinese"
docs = retreiver.invoke(query)

In [7]:
docs[0].page_content[:100]

'China and India maintained peaceful relations for thousands of years, but their relationship has var'

## Vector Store Retriever

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.schema import Document

In [ ]:
embeddings = HuggingFaceEmbeddings(
                model_name="BAAI/bge-small-en-v1.5",
                model_kwargs={"device": "cpu"},
                encode_kwargs={"normalize_embeddings": True}
            )

In [10]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [12]:
vector_store = Chroma.from_documents(
    documents=documents, embedding=embeddings, collection_name="sample"
)

In [13]:
retreiver = vector_store.as_retriever(search_kwargs={"k": 2})

In [14]:
query = "What is chroma is used for?"
docs = retreiver.invoke(query)

In [15]:
docs

[Document(id='1bf9b867-99c5-4b67-9d7c-3c0ce7f04a05', metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(id='4315b29a-d45c-46cc-9622-c320e2368061', metadata={}, page_content='LangChain helps developers build LLM applications easily.')]

## MMR

In [16]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [17]:
from langchain_community.vectorstores import FAISS

In [18]:
vector_store = FAISS.from_documents(
    documents=docs, embedding=embeddings
)

In [19]:
# Enable MMR in the retriever
retriever = vector_store.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [20]:
query = "What is Chroma?"
docs = retriever.invoke(query)
docs

[Document(id='465a07ae-e9b9-4004-9026-cf00db5dbfeb', metadata={}, page_content='Chroma is used to store and search document embeddings.'),
 Document(id='d085a56f-c138-4c69-8749-83d00a81b750', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(id='6bae85aa-425d-44e3-b45d-b867cdf4e2d1', metadata={}, page_content='MMR helps you get diverse results when doing similarity search.')]

## MultiQueryRetriever
The `MultiQueryRetriever` automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query

In [21]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [22]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [23]:
vector_store = FAISS.from_documents(
    documents=all_docs, embedding=embeddings
)
similarity_retreiver = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [24]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="openai/gpt-oss-20b")

In [26]:
multiquery_retreiver = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    llm=llm
)

In [27]:
# Query
query = "How to improve energy levels and maintain balance?"

In [28]:
# Retrieve results
similarity_results = similarity_retreiver.invoke(query)
multiquery_results= multiquery_retreiver.invoke(query)

In [29]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 5 ---
Deep sleep is crucial for cellular repair and emotional regulation.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional regula

## ContextualCompressionRetriever
👉 In simple words:

It takes results from another retriever (like a VectorStoreRetriever) and then compresses or filters the documents so that only the most relevant chunks remain before sending them to the LLM.

This helps when:

- Your retriever pulls too many or too long documents.

- You want to reduce token usage by summarizing or compressing irrelevant text.

- You need to ensure the LLM only sees highly relevant context.

In [30]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [31]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [33]:
# Create a FAISS vector store from the documents
vectorstore = FAISS.from_documents(docs, embeddings)

In [34]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [35]:
compressor = LLMChainExtractor.from_llm(llm)

In [36]:
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

In [37]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compressor_retriever.invoke(query)

In [38]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Result 3 ---
Photosynthesis does not occur in animal cells.
